In [ ]:
import sys
import time 
import vrep 
import numpy as np
from tank import *
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [ ]:
distance = ctrl.Antecedent(np.arange(0, 8.1, 0.1), 'distance')
speed = ctrl.Consequent(np.arange(0, 16, 1), 'speed')

behavior = "aggressive" # 'aggressive' or 'gentle'

if behavior == "aggressive":
    distance['very_close'] = fuzz.trapmf(distance.universe, [0, 0, 0.8, 1.2])
    distance['close'] = fuzz.trimf(distance.universe, [1.0, 1.5, 2.0])
    distance['medium'] = fuzz.trimf(distance.universe, [1.8, 3.0, 4.5])
    distance['far'] = fuzz.trapmf(distance.universe, [4.0, 5.0, 8.0, 8.0])
    
    speed['stop'] = fuzz.trapmf(speed.universe, [0, 0, 1, 3])
    speed['slow'] = fuzz.trimf(speed.universe, [2.5, 4, 7])
    speed['medium'] = fuzz.trimf(speed.universe, [6, 9, 12])
    speed['fast'] = fuzz.trapmf(speed.universe, [11, 13, 15, 15])
    
    rule1 = ctrl.Rule(distance['very_close'], speed['stop'])
    rule2 = ctrl.Rule(distance['close'], speed['slow'])
    rule3 = ctrl.Rule(distance['medium'], speed['medium'])
    rule4 = ctrl.Rule(distance['far'], speed['fast'])
    
else:  # gentle 

    distance['very_close'] = fuzz.trapmf(distance.universe, [0, 0, 1.0, 1.4])
    distance['close'] = fuzz.trimf(distance.universe, [1.2, 2.0, 3.0])
    distance['medium'] = fuzz.trimf(distance.universe, [2.5, 4.0, 5.5])
    distance['far'] = fuzz.trapmf(distance.universe, [5.0, 6.0, 8.0, 8.0])
    
    speed['stop'] = fuzz.trapmf(speed.universe, [0, 0, 1, 2])
    speed['slow'] = fuzz.trimf(speed.universe, [1, 3, 6])
    speed['medium'] = fuzz.trimf(speed.universe, [5, 8, 11])
    speed['fast'] = fuzz.trapmf(speed.universe, [10, 12, 16, 16])
    
    rule1 = ctrl.Rule(distance['very_close'], speed['stop'])
    rule2 = ctrl.Rule(distance['close'], speed['slow'])
    rule3 = ctrl.Rule(distance['medium'], speed['medium'])
    rule4 = ctrl.Rule(distance['far'], speed['medium']) 
    
# Create control system
speed_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4])
speed_sim = ctrl.ControlSystemSimulation(speed_ctrl)

In [ ]:
vrep.simxFinish(-1) # closes all opened connections, in case any prevoius wasnt finished
clientID=vrep.simxStart('127.0.0.1',19999,True,True,5000,5) # start a connection

if clientID!=-1:
    print ("Connected to remote API server")
else:
    print("Not connected to remote API server")
    sys.exit("Could not connect")

#create instance of Tank
tank=Tank(clientID)

# get handle to proximity sensor
err_code,ps_handle = vrep.simxGetObjectHandle(clientID,"Proximity_sensor", vrep.simx_opmode_blocking)

tank.go()

t = time.time()
while (time.time()-t)<60: # 60 seconds of communitation
    #read values from proximity sensor
    err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,ps_handle,vrep.simx_opmode_streaming)
    
    if detectionState: # if the sensor detects an object
        
        distance = np.linalg.norm(detectedPoint)
        
        speed_sim.input['distance'] = distance
        speed_sim.compute()
        
        fuzzy_speed = speed_sim.output['speed']
        
        print(f"distance: {distance}, fuzzy speed: {fuzzy_speed}")
        
        if fuzzy_speed < 1:
            tank.stop()
        else:
            tank.forward(fuzzy_speed)
    

vrep.simxStopSimulation(clientID,vrep.simx_opmode_oneshot) # stop the simulation in vrep

Connected to remote API server


1